<a href="https://colab.research.google.com/github/kabeerbora/ASI_Maps/blob/main/ASI_unit_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up environment

In [6]:
!pip -q install pyreadstat
import pyreadstat
import pandas as pd
from google.colab import drive
from glob import glob
import os
from tqdm import tqdm
import numpy as np
import geopandas as gpd
import subprocess
drive.mount("/gdrive")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 23.0 MB/s eta 0:00:00
Mounted at /gdrive


In [3]:
import warnings
warnings.filterwarnings("ignore")

[Pyreadstat docs](https://github.com/Roche/pyreadstat?tab=readme-ov-file)

## Read Index File

[Reading google sheets directly to pandas dataframe](https://gist.github.com/ivansaul/28257e793ae9b8575bb5c0f1b3906dad)

In [4]:
years = ['1999_2000','2000_2001','2001_2002','2002_2003','2003_2004','2004_2005','2005_2006','2006_2007','2007_2008','2008_2009','2009_2010']

In [7]:
%%time
#get spreadsheets key from url
gsheetkey = "1ogO9dkkgpr-kTcjYe__Yvw7-YiE6hFUXVAD2e-H2Olw"

#sheet name
sheet_name = 'Sheet1'

url=f'https://docs.google.com/spreadsheet/ccc?key={gsheetkey}&output=xlsx'
df_index = pd.read_excel(url,sheet_name=sheet_name, index_col=0)
df_index = df_index[~df_index['alias'].isna()]
# df_index['filename'] = df_index['filename'].str.strip().str.replace(r"[ ]+", "", regex=True)
df_index['year'] = df_index['year'].str.strip()
df_index = df_index[df_index["year"].isin(years)]
df_index.sample(5)

CPU times: user 1.42 s, sys: 74.1 ms, total: 1.5 s
Wall time: 3.24 s


,col_name,col_label,filename,year,alias,emp_labour,emp_labour_alias,sno,sno_values
1513.0,C_Itm3,Opening as on - Gross Value( Rs.),C-FIXED ASSETS,2003_2004,opening_gross_value,NaN,NaN,NaN,NaN
1663.0,E_Itm8,Wages/salaries (in Rs.),E-EMPLOYMENT AND LABOUR COST,2004_2005,wages_total,NaN,NaN,NaN,NaN
1970.0,DSL,Dispatch schedule no.,BLOCK-J,2006_2007,dsl,NaN,NaN,NaN,NaN
2113.0,A_Itm5,"Ind Code (5-digit, NIC-08)",A-IDENTIFICATION PARTICULARS,2008_2009,nic_code,NaN,NaN,NaN,NaN
1322.0,DSL,Dispatch Serial No,H-INPUT ITEMS INDIGENOUS,2001_2002,dsl,NaN,NaN,NaN,NaN


## Functions

In [9]:
def filter_E_by_sno(df, sno, sno_alias):
    cols = [i for i in df.columns if i not in ['dsl', 'sno']]
    xdf = df[df['sno']==sno][['dsl']+cols]
    new_cols = [f"{c}_{sno_alias}" for c in cols]
    xdf.columns = ["dsl"] + new_cols
    return xdf

def generate_emp_lab_dict(emp_lab_i, emp_lab_alias):

    column_index = eval(f"[{emp_lab_i}]")
    e_alias = eval(f"[{emp_lab_alias}]")

    d = dict(zip(column_index, e_alias))
    return d

def pivot_emp_labour_data(df, emp_labour_dict):
    final_df = None

    for i, (sx,sv) in enumerate(emp_labour_dict.items()):
        xdf = filter_E_by_sno(df, sx, sv)
        if i==0:
            final_df = xdf
        else:
            final_df = pd.merge(
                left=final_df,
                right=xdf,
                on='dsl',
                how="right"
            )

    return final_df

## Scan files


In [11]:
%%time

dir = r"/gdrive/MyDrive/data_economics_research/ASI_Firms"

dfs = {}

for (year,filename), dfx in tqdm(df_index.groupby(['year', 'filename'])):
    filepath = os.path.join(dir, year, filename+".sav")

    if os.path.isfile(filepath):

        filename = os.path.basename(filepath).split('.')[0].strip()
        year = os.path.basename(os.path.dirname(filepath))

        if dfx.shape[0] > 0:

            column_labels = list(dfx['col_name'].values)
            new_col_labels = list(dfx['alias'].values)
            col_mapper = dict(zip(column_labels, new_col_labels))

            dfy, meta_y = pyreadstat.read_sav(filepath, apply_value_formats=False, usecols=column_labels)
            dfy = dfy.rename(columns=col_mapper)

            ## Strip all column names and lower case them to maintain standardization
            dfy.columns = [c.strip().lower() for c in dfy.columns]

            if 'sno' in dfy.columns:
                dfy['sno'] = dfy['sno'].astype(int)

            # if 'scheme_code' in dfy.columns:
            #     dfy['scheme_code'] = dfy['scheme_code'].astype(int)

            #     ## Filter census only data
            #     dfy = dfy[dfy['scheme_code']==1]

            if "itm_code" in new_col_labels:
                dfy = dfy.loc[dfy.groupby("dsl")['itm_code'].transform("max")==dfy['itm_code']].copy()
                dfy.drop('itm_code', axis=1, inplace=True)

            sno_values = dfx.loc[(dfx['filename']==filename) & (~dfx['sno'].isna()), 'sno'].values

            ## Change the data type for serial number to Int:
            if len(sno_values)==1:
                sno = sno_values[0].astype(int)
            else:
                sno = None

            ## update metadata for each dataframe
            dfy.attrs['sno'] = sno
            dfy.attrs['filename'] = filename
            dfy.attrs['year'] = year

            ## Set the DSL data type to int
            dfy['dsl'] = dfy['dsl'].astype(int)

            ## Check if there are any additional notes for employment and labour
            if dfx['emp_labour'].notna().sum()==1:

                ## Then this is employee and labour data
                emp_labour_i = dfx.loc[dfx['alias']=='sno', "emp_labour"].iloc[0]
                emp_labour_v = dfx.loc[dfx['alias']=='sno', "emp_labour_alias"].iloc[0]

                ## Generate dict
                emp_labour_dict = generate_emp_lab_dict(emp_labour_i, emp_labour_v)

                ## Process employer and labour data
                dfy = pivot_emp_labour_data(dfy, emp_labour_dict)

            ## Store the dataframes into respective years
            if year in dfs.keys():
                dfs[year].append(dfy)
            else:
                dfs.setdefault(year, [dfy])

100%|██████████| 99/99 [00:55<00:00,  1.78it/s]

CPU times: user 22.5 s, sys: 1.85 s, total: 24.3 s
Wall time: 55.5 s


Check if all files have come through for each year

In [12]:
print("Number of dataframes per year:")
for year in years:
    print(year," | ",len(dfs[year]))

Number of dataframes per year:
1999_2000  |  9
2000_2001  |  9
2001_2002  |  9
2002_2003  |  9
2003_2004  |  9
2004_2005  |  9
2005_2006  |  9
2006_2007  |  9
2007_2008  |  9
2008_2009  |  9
2009_2010  |  9


## Compare DSL for each year

In [10]:
DSL_compare = {}

DSL_dfC = {}

for i in range(len(years)-1):

    dfA = dfs[years[i]][0]
    dfC = dfs[years[i]][2]
    dfC = dfC.loc[(dfC['sno']==dfC.attrs['sno']) & (dfC['closing_net_value']>0)].copy()
    dfC.dropna(subset=['closing_net_value'],inplace=True)

    df_merge = pd.merge(
        left=dfA,
        right=dfC,
        on='dsl',
        how='inner'
    )

    df_merge['ID'] = df_merge[['closing_net_value']].astype(int).astype(str).agg("_".join, axis=1)
    df_merge = df_merge[['ID', 'dsl', 'district_code', "nic_code"]]

    dfA2 = dfs[years[i+1]][0]
    dfC2 = dfs[years[i+1]][2]
    dfC2 = dfC2.loc[(dfC2['sno']==dfC2.attrs['sno']) & (dfC2['closing_net_value']>0)].copy()

    df_merge2 = pd.merge(
        left=dfA2,
        right=dfC2,
        on='dsl',
        how='inner'
    )

    df_merge2['ID'] = df_merge2[['opening_net_value']].astype(int).astype(str).agg("_".join, axis=1)
    df_merge2 = df_merge2[['ID', 'dsl', 'district_code', "nic_code"]]


    # Merge all
    df_dsl = pd.merge(
        left=df_merge,
        right=df_merge2,
        on='ID',
        how='inner',
        suffixes = ("_y1","_y2")
    )

    DSL_compare[f"{years[i]} to {years[i+1]}"] = df_dsl
    DSL_dfC[f"{years[i]} to {years[i+1]}"] = [dfC, dfC2]

In [11]:
for k,v in DSL_compare.items():
    print(k,"|",v.shape[0])

1999_2000 to 2000_2001 | 2
2000_2001 to 2001_2002 | 146
2001_2002 to 2002_2003 | 4154
2002_2003 to 2003_2004 | 4454
2003_2004 to 2004_2005 | 4629
2004_2005 to 2005_2006 | 6433
2005_2006 to 2006_2007 | 9645
2006_2007 to 2007_2008 | 11097
2007_2008 to 2008_2009 | 6398
2008_2009 to 2009_2010 | 6006


## Merge the dataframes


Merge the dataframes year wise and then concat into one single dataframe


In [13]:
%%time

all_dfs = []
errors = []

for year, dfz in dfs.items():

    dfi = None
    for i, dfxx in enumerate(dfz):

        ## Select by sno if any
        if 'sno' in dfxx.attrs.keys():
            sno = dfxx.attrs['sno']
        else:
            sno = None

        if sno is not None:
            dfxx = dfxx.loc[dfxx['sno']==sno, :].copy()
            dfxx.drop("sno", axis=1, inplace=True)

        ## Check for unique DSL per dataframe
        if len(dfxx['dsl'].unique())==dfxx.shape[0]:

            if i==0:
                dfi = dfxx
                dfi['year'] = year
            else:
                dfi = pd.merge(
                    left=dfi,
                    right=dfxx,
                    on=['dsl'],
                    how='left'
                )
        else:
            error_message = f"""
            Unique DSL not found for the year: {year}
            file: {dfxx.attrs['filename']}.
            """

            errors.append(error_message)

    all_dfs.append(dfi)

print()
print("".join(errors))



CPU times: user 1.48 s, sys: 121 ms, total: 1.6 s
Wall time: 1.63 s


**Concat all the dataframes together**

In [37]:
df_final
df_final_year = df_final.copy()
df_final_year['year'] = df_final_year['year'].str.split('_').str[1]
df_final_year['year'] = df_final_year['year'].astype(int)

In [16]:
df_final = pd.concat(all_dfs).reset_index(drop=True)
df_final

,dsl,scheme_code,nic_code,state_code,district_code,sector_code,year,ownership,opening_gross_value,closing_gross_value,...,total,rent_royalties_x,rent_rcvd_bldg,rent_royalties_y,int_received,rent_rcvd_land,gross_value,bonus,prov_fund,wel_exp
0,10001,1,24130.0,5,1.0,1.0,1999_2000,1.0,838612606.0,817821549.0,...,28931786.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10002,1,15510.0,5,9.0,1.0,1999_2000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10003,1,15510.0,5,9.0,1.0,1999_2000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10004,1,15511.0,5,14.0,1.0,1999_2000,6.0,218370918.0,481196866.0,...,24970778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10005,1,15420.0,5,23.0,2.0,1999_2000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557451,95318,2.0,45200.0,33.0,15.0,2.0,2009_2010,6.0,0.0,0.0,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,12865.0,0.0,17858.0
557452,95319,2.0,23960.0,33.0,28.0,2.0,2009_2010,6.0,46631814.0,31243988.0,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,193020.0,287354.0,55200.0
557453,95320,2.0,23960.0,33.0,30.0,1.0,2009_2010,6.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2580450.0,1674127.0,1213579.0
557454,95321,2.0,82920.0,33.0,11.0,2.0,2009_2010,6.0,0.0,0.0,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,210006.0,32826.0


In [39]:
df_merged = pd.merge(df_final_year, df_match, left_on=['year', 'wages_total_T'], right_on=['yr', 'ws'], how='inner')
df_merged

,dsl,scheme_code,nic_code,state_code_x,district_code,sector_code,year,ownership_x,opening_gross_value,closing_gross_value,...,ALL_35years,ALL_510years,ALL_over10years,ALL_totalcases,courtefficiency1,courtefficiency2,court1,court2,logrentpmfc,open
0,10001,1,24130.0,5,1.0,1.0,2000,1.0,838612606.0,817821549.0,...,13280.0,7251.0,2660.0,38643.0,0.119335,0.074295,0.0,1.0,9.210441,1.0
1,10004,1,15511.0,5,14.0,1.0,2000,6.0,218370918.0,481196866.0,...,13280.0,7251.0,2660.0,38643.0,0.119335,0.074295,0.0,1.0,11.356985,1.0
2,10006,1,35990.0,5,1.0,2.0,2000,4.0,46091711.0,46741710.0,...,13280.0,7251.0,2660.0,38643.0,0.119335,0.074295,0.0,1.0,10.698469,1.0
3,10010,1,15421.0,5,18.0,1.0,2000,5.0,109667255.0,212583211.0,...,13280.0,7251.0,2660.0,38643.0,0.119335,0.074295,0.0,1.0,10.701490,1.0
4,10014,1,15421.0,5,11.0,1.0,2000,6.0,133512397.0,152672333.0,...,13280.0,7251.0,2660.0,38643.0,0.119335,0.074295,0.0,1.0,10.884554,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282422,525317,2.0,25119,34.0,4.0,1.0,2008,6.0,13971685.0,14188393.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
282423,525318,2.0,27141,34.0,4.0,1.0,2008,6.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
282424,525319,2.0,27152,34.0,4.0,1.0,2008,6.0,133338539.0,148980089.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
282425,525320,2.0,27141,34.0,4.0,1.0,2008,6.0,65117477.0,86999008.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0


In [47]:
df_unique_fact_by_year = df_match.groupby('yr')['fact_id'].nunique()
df_unique_fact_by_year
df_unique_dsl_by_year = df_merged.groupby('year')['dsl'].nunique()
df_counts_factdsl = pd.merge(df_unique_fact_by_year, df_unique_dsl_by_year, left_index=True, right_index=True)
df_unique_fact_by_year = df_match.groupby('yr')['ws'].nunique()

,ws
yr,
1999.0,19391
2000.0,22474
2001.0,28520
2002.0,31236
2003.0,31707
2004.0,42216
2005.0,36788
2006.0,40141
2007.0,40236


In [132]:
df_merged_rows_by_year = df_merged.groupby('year').size()
df_merged_rows_by_year

,0
year,
2000,4451
2001,10774
2002,11766
2003,8567
2004,8520
2005,10014
2006,16143
2007,18657
2008,20813


In [140]:
df_merged['identifier'] = df_merged['dsl'].astype(str) + '_' + df_merged['year'].astype(str)

In [143]:
duplicate_identifiers = df_merged[df_merged.duplicated(subset=['identifier'], keep=False)]

print("Duplicate identifiers found:")
duplicate_identifiers

,dsl,scheme_code,nic_code,state_code_x,district_code,sector_code,year,ownership_x,opening_gross_value,closing_gross_value,...,ALL_510years,ALL_over10years,ALL_totalcases,courtefficiency1,courtefficiency2,court1,court2,logrentpmfc,open,identifier
135,10411,1,17111.0,26,8.0,2.0,2000,6.0,40889470.0,40889965.0,...,7251.0,2660.0,38643.0,0.119335,0.074295,0.0,1.0,0.000000,1.0,10411_2000
136,10411,1,17111.0,26,8.0,2.0,2000,6.0,40889470.0,40889965.0,...,7251.0,2660.0,38643.0,0.119335,0.074295,0.0,1.0,0.000000,1.0,10411_2000
137,10411,1,17111.0,26,8.0,2.0,2000,6.0,40889470.0,40889965.0,...,6237.0,1622.0,40390.0,0.106159,0.224833,1.0,1.0,0.000000,1.0,10411_2000
138,10411,1,17111.0,26,8.0,2.0,2000,6.0,40889470.0,40889965.0,...,10705.0,2235.0,90384.0,0.369565,0.424655,0.0,0.0,13.575367,1.0,10411_2000
139,10411,1,17111.0,26,8.0,2.0,2000,6.0,40889470.0,40889965.0,...,22991.0,3133.0,94454.0,0.024481,0.036992,1.0,1.0,0.000000,1.0,10411_2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109687,45451,1,23101,18.0,14.0,1.0,2008,6.0,0.0,619571.0,...,684.0,241.0,14343.0,0.454268,0.582444,0.0,0.0,0.000000,1.0,45451_2008
109688,45451,1,23101,18.0,14.0,1.0,2008,6.0,0.0,619571.0,...,6110.0,578.0,73403.0,0.099102,0.304293,1.0,1.0,0.000000,1.0,45451_2008
109689,45451,1,23101,18.0,14.0,1.0,2008,6.0,0.0,619571.0,...,6110.0,578.0,73403.0,0.099102,0.304293,1.0,1.0,0.000000,1.0,45451_2008
109690,45451,1,23101,18.0,14.0,1.0,2008,6.0,0.0,619571.0,...,3638.0,480.0,33834.0,0.250216,0.229473,0.0,1.0,0.000000,1.0,45451_2008


In [2]:
df_new = duplicate_identifiers[['identifier', 'fact_id', 'ws', 'wages_total_T']].copy()
filtered_df = df_new[df_new['identifier'] == '10191_2008']
print("Filtered DataFrame:")
print(filtered_df)

NameError: name 'df_new' is not defined

In [157]:
df_merged = df_merged.drop_duplicates(subset=['identifier'])
df_merged.to_excel('/gdrive/MyDrive/data_economics_research/ASI_Firms/common_ids.xlsx', index=False)

In [1]:
rows_with_equal_state_codes = df_merged[df_merged['state_code_x'] == df_merged['state_code']].shape[0]
print(f"Number of rows with equal state_code_x and state_code: {rows_with_equal_state_codes}")

NameError: name 'df_merged' is not defined

**Check for percentage not null values in each field:**

In [14]:
df_final.notna().sum()/df_final.shape[0]*100

,0
dsl,100.000000
scheme_code,100.000000
nic_code,100.000000
state_code,100.000000
district_code,100.000000
sector_code,100.000000
year,100.000000
ownership,87.154467
opening_gross_value,73.648190
closing_gross_value,73.648190


In [ ]:
total_non_na_pur_value_imp = df_final['pur_value_imp'].notna().sum()
print(f"Total number of non-NA values in pur_value_imp: {total_non_na_pur_value_imp}")

Total number of non-NA values in pur_value_imp: 42917


In [ ]:
df_final.columns

Index(['dsl', 'scheme_code', 'nic_code', 'state_code', 'district_code',
       'sector_code', 'year', 'ownership', 'opening_gross_value',
       'closing_gross_value', 'opening_net_value', 'closing_net_value',
       'man_total_C', 'wages_total_C', 'bonus_C', 'prov_fund_C', 'wel_exp_C',
       'man_total_TW', 'wages_total_TW', 'bonus_TW', 'prov_fund_TW',
       'wel_exp_TW', 'man_total_SM', 'wages_total_SM', 'bonus_SM',
       'prov_fund_SM', 'wel_exp_SM', 'man_total_T', 'wages_total_T', 'bonus_T',
       'prov_fund_T', 'wel_exp_T', 'op_expenses', 'total_exp',
       'rent_paid_bldg', 'rent_paid_land', 'int_paid', 'tot_receipts',
       'pur_value_ind', 'pur_value_imp', 'gross_sale', 'excise_duty',
       'sales_tax', 'others_product', 'total', 'rent_royalties_x',
       'rent_rcvd_bldg', 'rent_royalties_y', 'int_received', 'rent_rcvd_land',
       'gross_value', 'bonus', 'prov_fund', 'wel_exp'],
      dtype='object')

## Export the Aggregate

Before exporting the dataset, make sure all the values are converted to integer to store it efficiently

In [ ]:
%%time

## Convert all the numeric columns to integer type
numeric_cols = [c for c in df_final.columns if pd.api.types.is_numeric_dtype(df_final[c])]
df_final = df_final[numeric_cols].fillna(0, axis=1)
df_final[numeric_cols] = df_final[numeric_cols].astype(int)

df_final.to_parquet(r"/gdrive/MyDrive/Projects/Annual Survey of India/Data/ASI_aggregate.parquet")

CPU times: user 894 ms, sys: 243 ms, total: 1.14 s
Wall time: 1.28 s


In [ ]:
%%time
df_final.to_parquet(r"/gdrive/MyDrive/Projects/Annual Survey of India/Data/ASI_aggregate.parquet")

## Evaluation

In [15]:
for year in years:
    df_C = dfs[year][2]

In [87]:
df_A = df_
df_merged = pd.merge(df_A, df_E, on='dsl', how='inner')
df_A

,Year,Block code 'A',Despatch Serial No,Scheme code,"Ind Code as per Return (5-digit, NIC-98)",State Code,District code,Rural/Urban code,No. of units,Status of Unit (Code 17 to 20 Extracted data from ASI 01-02),Number of working days ( Manufacturing days),Number of working days (Non-Manufacturing days),Number of working days ( Total),Cost of Production,Inflation/Multiplier factor (in 9999.9999 format)


In [93]:
df_E = dfs["2002_2003"][3]
df_A = dfs["2002_2003"][0]
df_merged = pd.merge(df_A, df_E, on='dsl', how='inner')
df_merged

,dsl,scheme_code,nic_code,state_code,district_code,sector_code,year,man_total_C,wages_total_C,bonus_C,...,man_total_SM,wages_total_SM,bonus_SM,prov_fund_SM,wel_exp_SM,man_total_T,wages_total_T,bonus_T,prov_fund_T,wel_exp_T
0,10001,1,20101.0,13.0,1.0,1.0,2002_2003,NaN,NaN,NaN,...,290.0,16240.0,0.0,0.0,0.0,2568.0,132418.0,0.0,0.0,560.0
1,10002,1,20101.0,13.0,1.0,1.0,2002_2003,NaN,NaN,NaN,...,298.0,21370.0,0.0,0.0,0.0,1743.0,73896.0,0.0,0.0,350.0
2,10003,1,20211.0,13.0,1.0,1.0,2002_2003,NaN,NaN,NaN,...,6900.0,5708032.0,308235.0,0.0,0.0,80480.0,15972340.0,862769.0,1286528.0,733954.0
3,10004,1,20211.0,13.0,1.0,1.0,2002_2003,40613.0,1624500.0,0.0,...,600.0,151160.0,0.0,0.0,0.0,61296.0,3058570.0,0.0,118420.0,19650.0
4,10005,1,20211.0,13.0,1.0,1.0,2002_2003,NaN,NaN,NaN,...,1665.0,258080.0,0.0,0.0,0.0,8181.0,575842.0,0.0,0.0,104199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11011,24803,1,20101.0,35.0,1.0,2.0,2002_2003,NaN,NaN,NaN,...,305.0,192000.0,0.0,0.0,0.0,4264.0,588435.0,0.0,13244.0,14810.0
11012,24804,1,15124.0,35.0,1.0,1.0,2002_2003,NaN,NaN,NaN,...,276.0,97573.0,0.0,0.0,0.0,8479.0,938037.0,0.0,114213.0,13423.0
11013,24805,1,15142.0,35.0,1.0,1.0,2002_2003,NaN,NaN,NaN,...,1196.0,615366.0,0.0,156519.0,3395.0,27484.0,3961103.0,0.0,1055783.0,82632.0
11014,24806,1,24123.0,28.0,14.0,2.0,2002_2003,96298.0,19249591.0,0.0,...,109436.0,122567802.0,0.0,0.0,0.0,421521.0,238911355.0,2115765.0,24231561.0,30837103.0


In [17]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
import io

for fn in uploaded.keys():
  df_match = pd.read_stata(io.BytesIO(uploaded[fn]))

Saving ACN_final_data_submission.dta to ACN_final_data_submission.dta


In [29]:
duplicate_ws_counts = df_match['ws'].value_counts()
duplicate_ws = duplicate_ws_counts[duplicate_ws_counts > 1].index.tolist()
df_match = df_match.drop_duplicates(subset=['ws'], keep='first')
df_match

,count
ws,
607591.0,1
1149558.0,1
51497096.0,1
375665.0,1
822029.0,1
...,...
23646046.0,1
7319476.0,1
38067328.0,1


In [31]:
df_match = df_match.dropna(subset=['ws'])  # Remove rows with missing 'ws' values
df_match = df_match[df_match['ws'] != 0]

,YR,fact_id,NoF,openClosed,yr,notnumeric,org,ownership,unitsCompany,year_init_prodn,...,ALL_35years,ALL_510years,ALL_over10years,ALL_totalcases,courtefficiency1,courtefficiency2,court1,court2,logrentpmfc,open
0,2005.0,0542322F,1.0,Open,2005.0,0.0,Individual Proprietorship,Wholly Private Ownership,0.0,1994.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
1,2005.0,0800720F,1.0,Open,2005.0,0.0,Public Limited Company,Wholly Private Ownership,1.0,1995.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.408271,1.0
2,2005.0,0130919F,1.0,Open,2005.0,0.0,Public Limited Company,Wholly Private Ownership,3.0,1939.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
3,1999.0,0656322F,1.0,Open,1999.0,0.0,Public Limited Company,Wholly Private Ownership,NaN,1997.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.335833,1.0
4,2005.0,0235023F,1.0,Open,2005.0,0.0,Private Limited Company,Wholly Private Ownership,10.0,1984.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354413,2002.0,0095719F,1.0,Open,2002.0,0.0,Public Limited Company,Wholly Private Ownership,1.0,1975.0,...,5241.0,2963.0,410.0,20373.0,0.112633,0.19143,1.0,1.0,0.000000,1.0
354414,2004.0,1054919F,2.0,Open,2004.0,0.0,Individual Proprietorship,Wholly Private Ownership,0.0,1999.0,...,5241.0,2963.0,410.0,20373.0,0.112633,0.19143,1.0,1.0,11.787530,1.0
354415,2003.0,0756319F,1.0,Open,2003.0,0.0,Individual Proprietorship,Wholly Private Ownership,0.0,1973.0,...,5241.0,2963.0,410.0,20373.0,0.112633,0.19143,1.0,1.0,0.000000,1.0
354416,2005.0,0165919F,1.0,Open,2005.0,0.0,Public Limited Company,Wholly Private Ownership,1.0,1935.0,...,5241.0,2963.0,410.0,20373.0,0.112633,0.19143,1.0,1.0,0.000000,1.0


In [18]:
df_match

,YR,fact_id,NoF,openClosed,yr,notnumeric,org,ownership,unitsCompany,year_init_prodn,...,ALL_35years,ALL_510years,ALL_over10years,ALL_totalcases,courtefficiency1,courtefficiency2,court1,court2,logrentpmfc,open
0,2005.0,0542322F,1.0,Open,2005.0,0.0,Individual Proprietorship,Wholly Private Ownership,0.0,1994.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
1,2005.0,0800720F,1.0,Open,2005.0,0.0,Public Limited Company,Wholly Private Ownership,1.0,1995.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.408271,1.0
2,2005.0,0130919F,1.0,Open,2005.0,0.0,Public Limited Company,Wholly Private Ownership,3.0,1939.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
3,1999.0,0656322F,1.0,Open,1999.0,0.0,Public Limited Company,Wholly Private Ownership,NaN,1997.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.335833,1.0
4,2005.0,0235023F,1.0,Open,2005.0,0.0,Private Limited Company,Wholly Private Ownership,10.0,1984.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354421,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,8.0,NaN,0.250000,0.0,0.0,NaN,0.0
354422,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.0,3.0,0.0,2086.0,0.871492,0.890221,0.0,0.0,NaN,0.0
354423,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,285.0,112.0,0.0,2964.0,0.052632,0.660256,1.0,1.0,NaN,0.0
354424,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,0.0,121.0,0.222222,0.636364,0.0,1.0,NaN,0.0


In [113]:
print(df_match.columns.tolist())

['YR', 'fact_id', 'NoF', 'openClosed', 'yr', 'notnumeric', 'org', 'ownership', 'unitsCompany', 'year_init_prodn', 'expenseRentPlant', 'expenseTotal', 'expenseRentBuilding', 'expenseRentLand', 'state_code', 'state_name', 'current_assets', 'GVAFC', 'rent_paid', 'avg_num_persons', 'workers', 'men', 'women', 'children', 'directly_emp', 'sup_mang_staff', 'other_emp', 'mandays_man', 'mandays_nonman', 'mandays_total', 'ws_workers', 'ws_sup_mang_staff', 'ws_otheremp', 'ws', 'total_assets', 'tangibility1', 'tertile1', 'tertile2', 'tertile3', 'tertile4', 'tertile5', 'tertile6', 'tang3_mean', 'tang_tercile_vig', 'total_emp', 'measure1', 'measure2', 'measure3', 'law', 'treatment1', 'treatment2', 'treatment3', 'posttreat1', 'posttreat2', 'posttreat3', 'weight', 'profit_by_tot_assets', 'profit_by_totass', 'log_total_assets', 'ln_totass', 'tot_worker', 'sttradecredit1', 'STtradecredit', 'ln_STtradecredit', 'STtradecreditbytotass1', 'STtradecreditbytotass', 'STformalcredit1', 'STformalcredit', 'ln_STf

In [96]:
df_E_wages_total_TW = df_merged['wages_total_T']
df_match_2002_ws = df_match_2002['ws']
df_E_wages_total_TW.equals(df_match_2002_ws)
df_merged = pd.merge(df_A, df_E, on='', how='inner')

False

In [111]:
df_merged = pd.merge(df_merged, df_match_2002, left_on='wages_total_T', right_on='ws', how='inner')
df_match.columns

Index(['YR', 'fact_id', 'NoF', 'openClosed', 'yr', 'notnumeric', 'org',
       'ownership', 'unitsCompany', 'year_init_prodn',
       ...
       'ALL_35years', 'ALL_510years', 'ALL_over10years', 'ALL_totalcases',
       'courtefficiency1', 'courtefficiency2', 'court1', 'court2',
       'logrentpmfc', 'open'],
      dtype='object', length=137)

## Unique Factor ID

In [ ]:
df_path = rf"/gdrive/MyDrive/Projects/Annual Survey of India/ASI Unit Level/{year}/A-IDENTIFICATION PARTICULARS.sav"
df_A, meta_A = pyreadstat.read_sav(df_path, apply_value_formats=True)
df_A.columns = meta_A.column_labels

df_A['Scheme Code'].value_counts()

In [ ]:
df_C = dfs['2003_2004'][2]
df_C.loc[df_C['dsl']==10001]

In [ ]:
len(dfs['2002_2003'][2]['dsl'].unique()), len(dfs['2002_2003'][2])

In [ ]:
pd.merge(
    left=dfs['2002_2003'][0],
    right=dfs['2002_2003'][2],
    on='dsl',
    how='inner'
)